In [1]:
import hdbscan
import pandas as pd
import category_encoders
import numpy as np
import math
PREDICCION_REAL = True

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_pickle("../Archivos/Arboles_entrenamiento_exp.pkl").drop(columns = ["Opportunity_ID"])
test = pd.read_pickle("../Archivos/Arboles_validacion_exp.pkl").drop(columns = ["Opportunity_ID"])
test_real = pd.read_pickle("../Archivos/Arboles_validacion_exp.pkl")
if PREDICCION_REAL:
    test = pd.read_pickle("../Archivos/Arboles_test_exp.pkl").drop(columns = ["Opportunity_ID"])
    test_real = pd.read_pickle("../Archivos/Arboles_test_exp.pkl")

In [3]:
#FECHAS A DIAS

columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date',"Year-Month"]
def fecha_a_dias(x):
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)

In [4]:
objetivo = (entrenamiento['Stage'] == 1).astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_category = list(entrenamiento.select_dtypes(include=['category']).columns)
if 'Stage' in columnas_category : columnas_category.remove('Stage')

In [5]:
if not PREDICCION_REAL:
    test_label = (test['Stage'] == 1).astype(int)
    test = test.drop(columns=['Stage'])

In [6]:
def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['category']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['category']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
categoricas_a_numericas(entrenamiento,test,objetivo,False)
categoricas_a_numericas(entrenamiento,test,objetivo,True)

C:\Users\gianb\anaconda3\envs\hdbscan\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\gianb\anaconda3\envs\hdbscan\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [7]:
def limpiar_nan(col,entrenamiento,test):
    mean = entrenamiento[col].mean()
    entrenamiento[col] = entrenamiento[col].replace(np.NaN,mean)
    mean = test[col].mean()
    test[col] = test[col].replace(np.NaN,mean)
    
def limpiar_inf(col,entrenamiento,test):
    entrenamiento[col] = entrenamiento[col].replace(math.inf,np.NaN)
    test[col] = test[col].replace(math.inf,np.NaN)    

In [8]:
def limpiar(entrenamiento,test,limite):
    for col in entrenamiento.columns:
        ent_null = entrenamiento.loc[entrenamiento[col] == np.inf]
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_inf(col,entrenamiento,test)        
    for col in test.columns:
        ent_null = test.loc[test[col] == np.inf]
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_inf(col,entrenamiento,test)        

    for col in entrenamiento.columns:
        ent_null = entrenamiento.loc[entrenamiento[col].isnull()]
        ent_null = ent_null.replace(np.NaN,0)
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_nan(col,entrenamiento,test)        
    for col in test.columns:
        ent_null = test.loc[test[col].isnull()]
        ent_null = ent_null.replace(np.NaN,0)
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_nan(col,entrenamiento,test)

In [9]:
a_usar = ['Bureaucratic_Code',
 'Account_Name',
 'Opportunity_Name',
 'Opportunity_Owner',
 'Last_Modified_By',
 'Product_Family',
 'Product_Name',
 'Opportunity_Name_Planned_Time_Until_Deliver_mean',
 'Opportunity_Name_Planned_Opportunity_Duration_mean',
 'Product_Name_Planned_Time_Until_Deliver_mean',
 'Product_Name_Planned_Opportunity_Duration_mean',
 'Territory',
 'Account_Owner',
 'Account_Type',
 'Opportunity_Type',
 'Planned_Opportunity_Duration',
 'Bureaucratic_Code_ASP_(converted)_std',
 'Bureaucratic_Code_Total_Amount_mean',
 'Bureaucratic_Code_Total_Product_Family_Region_Last_Month_mean',
 'Bureaucratic_Code_Total_Product_Family_Region_Last_Month_std',
 'Bureaucratic_Code_Total_Products_Region_Last_Week_std',
 'Bureaucratic_Code_Total_Products_Region_Last_Month_mean',
 'Bureaucratic_Code_Planned_Deliver_Duration_mean',
 'Bureaucratic_Code_Planned_Deliver_Duration_std',
 'Bureaucratic_Code_Actual_Opportunity_Duration_std',
 'Bureaucratic_Code_Planned_Time_Until_Deliver_mean',
 'Bureaucratic_Code_Planned_Time_Until_Deliver_std',
 'Bureaucratic_Code_Planned_Opportunity_Duration_mean',
 'Bureaucratic_Code_Product_Amount_Deviation_of_Product_Family_rate_std',
 'Bureaucratic_Code_Opportunity_Duration_Ratio_std',
 'Bureaucratic_Code_Opportunity_Total_Amount_Region_avg_std',
 'Bureaucratic_Code_Opportunity_TRF_Region_std_Ratio_std',
 'Bureaucratic_Code_Opportunity_Duration_by_Account_Type_std',
 'Bureaucratic_Code_ASP_by_Billing_Country_mean_std',
 'Bureaucratic_Code_Opportunity_Duration_by_Product_Family_mean_mean',
 'Bureaucratic_Code_Buro_Approved_by_Product_Family_std',
 'Account_Name_Planned_Opportunity_Duration_mean',
 'Opportunity_Type_Planned_Time_Until_Deliver_mean',
 'Opportunity_Type_Opportunity_Duration_by_Account_Type_mean',
 'Last_Modified_By_Planned_Time_Until_Deliver_mean',
 'Last_Modified_By_Opportunity_Total_Amount_Region_avg_std',
 'Last_Modified_By_Opportunity_Total_Amount_Region_std_std',
 'Last_Modified_By_Opportunity_TRF_Region_avg_std',
 'Last_Modified_By_ASP_by_Region_mean_std',
 'Product_Family_Planned_Opportunity_Duration_mean',
 'Product_Name_Planned_Time_Until_Deliver_std',
 'Product_Name_Opportunity_Duration_by_Account_Type_mean',
 'Bureaucratic_Code_Billing_Country_unique',
 'Bureaucratic_Code_Opportunity_Name_unique',
 'Bureaucratic_Code_Opportunity_Type_unique',
 'Bureaucratic_Code_Total_Amount_Currency_unique',
 'Last_Modified_By_Territory_unique',
 'Last_Modified_By_Billing_Country_unique',
 'Last_Modified_By_Account_Name_unique',
 'Last_Modified_By_Account_Owner_unique',
 'Last_Modified_By_Account_Type_unique',
 'Last_Modified_By_Product_Name_unique',
 'Last_Modified_By_Total_Amount_Currency_unique',
 'Region',
 'Billing_Country']

drop = list(entrenamiento.columns)
for col in a_usar:
    drop.remove(col)
    
entrenamiento = entrenamiento.drop(columns = drop)

In [10]:
test = test.drop(columns = drop)

In [11]:
limpiar(entrenamiento,test,1000)

In [12]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, prediction_data=True).fit(entrenamiento)

In [13]:
import pprint
clusters = dict()
labels = set(clusterer.labels_)
for l in labels:
    cluster = [1 if i == l else 0 for i in clusterer.labels_]
    suma = 0
    for i in cluster:
        suma += i
    clusters[l] = suma
pprint.PrettyPrinter().pprint(clusters)    

{-1: 77, 0: 128, 1: 152, 2: 1501, 3: 1479, 4: 2288, 5: 6515}


In [14]:
entrenamiento_con_cluster = entrenamiento.copy()
entrenamiento_con_cluster["Cluster"] = clusterer.labels_
entrenamientos_separados = []
for label in labels:
    entrenamiento_cluster = entrenamiento_con_cluster.loc[entrenamiento_con_cluster["Cluster"] == label]
    entrenamientos_separados.append(entrenamiento_cluster)
entrenamientos_separados = [i.drop(columns = ["Cluster"]) for i in entrenamientos_separados] 

In [15]:
test_labels, strengths = hdbscan.approximate_predict(clusterer, test)
test_labels

array([4, 4, 4, ..., 4, 4, 3])

In [16]:
clusters = dict()
labels = set(clusterer.labels_)
for l in labels:
    cluster = [1 if i == l else 0 for i in test_labels]
    suma = 0
    for i in cluster:
        suma += i
    clusters[l] = suma
pprint.PrettyPrinter().pprint(clusters)    

{-1: 33, 0: 42, 1: 109, 2: 0, 3: 368, 4: 1995, 5: 4}
